In [1]:
# # connect to google drive
# from google.colab import drive
# drive.mount(r'/content/drive')

# # install package
# !pip install partd

In [68]:
import os
import dask.dataframe as dd
import pandas as pd
import numpy as np

project_root = r'C:\Users\howard810804\Desktop\raw_data'
columns_jp2en = {
    r'モニターCue ※納品不可': r'Monitor_ID',
    r'日時(yyyy-mm-dd hh:mm:ss)': r'Datetime',
    r'接触時間（duration)': r'Duration',
    r'OS（Android or iOS）': r'OS',
    r'コンテンツID': r'Content_ID',
    r'パッケージId': r'Package_ID',
    r'コンテンツ名': r'Content_Name',
    r'アプリカテゴリ': r'APP_Cate',
    r'アプリ提供企業名': r'APP_Provider',
    r'アプリ名': r'APP_Name'
}
selected_cate = [r'ツール類', r'ソーシャルネットワーキング', r'ゲーム']

func = lambda fd: os.path.isdir(fd)
folder_ls = filter(func, [os.path.join(project_root, fd) 
                          for fd in os.listdir(project_root)])

for fd in folder_ls:
    print(f'>> 正在處理 {fd} 資料夾裡的檔案')
    fl_ls = sorted(os.listdir(fd))[:2]
    
    df_log = dd.read_csv(os.path.join(fd, fl_ls[0]), sep='\t')  # 讀取access log裡面的資訊, 暫不捨棄任何columns
    df_log = df_log.rename(columns=columns_jp2en)  # rename dataframe columns from japanese to english

    # step 2: filter/remove the data we need/don't need
    df_log = df_log.loc[df_log[r'APP_Cate'].isin(selected_cate)]  # remove non top-3-categories rows
    df_log = df_log.dropna()  # remove those rows with any missing value
    print(df_log)
    
#     # read user profile from another dataframe
#     df_usr = dd.read_csv(os.path.join(fd, fl_ls[1]), sep='\t')  # 讀取user profile裡面的資訊, 暫不捨棄任何columns
#     df_usr = df_usr.dropna()  # remove those rows with any missing value
    
    

    # step 3: transform data to the form we need, 引入featuretools
#     # replace category name (str) with label number (int)
#     f = lambda cate: {cate[i]: i for i in range(len(cate))}
    
#     {
#             r'ツール類': 0,
#             r'ソーシャルネットワーキング': 1,
#             r'ゲーム': 2
#         }[cate]
#         df['Cate_Label'] = df['Cate_Label'].map(f, meta=('Cate_Label', int))

#         # generate date columns
#         df['Date'] = df['Timestamp'].map(lambda x: x[:10], meta=('Date', str))

#         # convert time (str dtype) to timestamp (int dtype), then divide 10^9
#         df['Timestamp'] = df['Timestamp'].astype('M8[us]').\
#             astype(np.int64) // 10 ** 9



#         # step 4: generate ret data as result
#         # split original df into 3 categories
#         dfc_ts, dfc_dr = {}, {}
#         f = lambda x: x.tolist()
    
#         for i in range(3):  # 因為是3個類別, 故range == 3
#             _dfc = df[df['Cate_Label'] == i]
#             _dfc_gr = _dfc.groupby(['Monitor', 'Date'])

#             dfc_ts[i] = _dfc_gr['Timestamp'].\
#                 apply(f, meta=('Timestamp', int))
#             dfc_dr[i] = _dfc_gr['Duration'].\
#                 apply(f, meta=('Duration', int))
            
#         return dfc_ts, dfc_dr
    
#     df_usr = dd.read_csv(os.path.join(fd, fl_ls[1]), sep='\t')
#     print(df_log.head(3))
#     print(df_usr.head(3))
                     
    # file_pa = os.path.join(folder_pa, r's35809_extract_condition_dl_list_104628.tsv')
# # use_cols = [r'モニターCue ※納品不可', r'日時(yyyy-mm-dd hh:mm:ss)',
# #             r'接触時間（duration)', r'アプリカテゴリ']
# df = pd.read_csv(file_pa, sep='\t', header=None, nrows=10)
# # dt = {
# #     r'モニターCue ※納品不可': np.int64,
# #     r'日時(yyyy-mm-dd hh:mm:ss)': str,
# #     r'接触時間（duration)': np.int64,
# #     r'アプリカテゴリ': str
# # }
# dt = df.dtypes.to_dict()
# df = dd.read_csv(file_pa, sep='\t', dtype=dt)


# # file_ls = [os.path.join(folder_pa, fl) for fl in os.listdir(folder_pa)]
# # file_ls[0]

# # df = dd.read_csv(r'C:\\Users\\howard810804\\Desktop\\raw_data\\s35809_MobileApp_201701_104628\\s35809_extract_condition_dl_list_104628.tsv', sep='\t')
# # df

>> 正在處理 C:\Users\howard810804\Desktop\raw_data\s35809_MobileApp_201701_104628 資料夾裡的檔案
Dask DataFrame Structure:
               Monitor_ID Datetime Duration      OS Content_ID Package_ID Content_Name APP_Cate APP_Provider APP_Name
npartitions=97                                                                                                       
                    int64   object    int64  object      int64     object       object   object       object   object
                      ...      ...      ...     ...        ...        ...          ...      ...          ...      ...
...                   ...      ...      ...     ...        ...        ...          ...      ...          ...      ...
                      ...      ...      ...     ...        ...        ...          ...      ...          ...      ...
                      ...      ...      ...     ...        ...        ...          ...      ...          ...      ...
Dask Name: dropna, 777 tasks
>> 正在處理 C:\Users\howard810804\Des

In [ ]:
df.head(10)

In [14]:
import featuretools as ft
data = ft.demo.load_mock_customer()

customers_df = data["customers"]
sessions_df = data["sessions"]
transactions_df = data["transactions"]

# 這邊不太懂
entities = {
    "customers": (customers_df, "customer_id"),
    "sessions": (sessions_df, "session_id", "session_start"),
    "transactions": (transactions_df, "transaction_id", "transaction_time")
}
relationships = [
    ("sessions", "session_id", "transactions", "session_id"),
    ("customers", "customer_id", "sessions", "customer_id")
]

feature_matrix_customers, features_defs = ft.dfs(entities=entities,
                                                 relationships=relationships,
                                                 target_entity="customers")

# feature_matrix_customers

features_defs

[<Feature: zip_code>,
 <Feature: COUNT(sessions)>,
 <Feature: NUM_UNIQUE(sessions.device)>,
 <Feature: MODE(sessions.device)>,
 <Feature: SUM(transactions.amount)>,
 <Feature: STD(transactions.amount)>,
 <Feature: MAX(transactions.amount)>,
 <Feature: SKEW(transactions.amount)>,
 <Feature: MIN(transactions.amount)>,
 <Feature: MEAN(transactions.amount)>,
 <Feature: COUNT(transactions)>,
 <Feature: NUM_UNIQUE(transactions.product_id)>,
 <Feature: MODE(transactions.product_id)>,
 <Feature: DAY(join_date)>,
 <Feature: DAY(date_of_birth)>,
 <Feature: YEAR(join_date)>,
 <Feature: YEAR(date_of_birth)>,
 <Feature: MONTH(join_date)>,
 <Feature: MONTH(date_of_birth)>,
 <Feature: WEEKDAY(join_date)>,
 <Feature: WEEKDAY(date_of_birth)>,
 <Feature: SUM(sessions.STD(transactions.amount))>,
 <Feature: SUM(sessions.MAX(transactions.amount))>,
 <Feature: SUM(sessions.SKEW(transactions.amount))>,
 <Feature: SUM(sessions.MIN(transactions.amount))>,
 <Feature: SUM(sessions.MEAN(transactions.amount))>,
 <

In [2]:
# import multiprocessing
# multiprocessing.set_start_method('forkserver')

from dask import dataframe as dd
from typing import List, Tuple
import os
import gzip
import json
import numpy as np
import pandas as pd
import partd
import time

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from tpot import TPOTClassifier
import xgboost

In [60]:
class ISSP:
#     def __init__(self, proj_config, tpot_config):
    def __init__(self):
#         # project config
# #         self._selected_cate = [r'ツール類', r'ソーシャルネットワーキング', r'ゲーム']
        
#         self._test_size = proj_config['test_size']
# #         self._tpot_args = {
# #             r'generations': 5,
# #             r'population_size': 250,
# #             r'verbosity': 3,
# #             r'cv': 5,
# #             r'subsample': 0.01,
# # #             r'n_jobs': -1
# #             r'n_jobs': 1
# #         }
#         self._clf = None
    
#         # user config
#         self._project_root = proj_config['project_root']
#         assert os.path.exists(self._project_root), r'[Error] path does not exist'

        self._proj_root = r'D:\issp_data'        
        
        def mkdir(pa):
            if not os.path.exists(pa): os.mkdir(pa)
        
        self._data_raw = os.path.join(self._proj_root, r'raw_tsdr')
        mkdir(self._data_raw)

        self._data_pretsdr = os.path.join(self._proj_root, r'pre_tsdr')
        mkdir(self._data_pretsdr)
        
        
    def preprocess_all(self):
        file_ls = os.listdir(self._data_raw)
        print(file_ls)
        
        for src_file_nm in file_ls:
            self._preprocess_tsdr(src_file_nm, self._data_pretsdr)


    def _preprocess_tsdr(self, src_file_nm: str, dest_folder_pa: str) -> None:
        _use_cols = [r'モニターCue ※納品不可', r'日時(yyyy-mm-dd hh:mm:ss)',
                     r'接触時間（duration)', r'アプリカテゴリ']
        _dtypes = {
            r'モニターCue ※納品不可': np.int64,
            r'日時(yyyy-mm-dd hh:mm:ss)': str,
            r'接触時間（duration)': np.int64,
            r'アプリカテゴリ': str
        }
        _selected_cates = [r'ツール類', r'ソーシャルネットワーキング', r'ゲーム']
        
        # step 1: load raw data from disk
        df = dd.read_csv(os.path.join(self._data_raw, src_file_nm), 
                         sep='\t', usecols=_use_cols, dtype=_dtypes)

        # rename dataframe columns from japanese to english
        df = df.rename(columns={
            r'モニターCue ※納品不可': r'Monitor_ID',
            r'日時(yyyy-mm-dd hh:mm:ss)': r'Timestamp',
            r'接触時間（duration)': r'Duration',
            r'アプリカテゴリ': r'Cate_Label'
        })

        # step 2: filter/remove the data we need/don't need
        # remove non top-3-categories rows
        df = df.loc[df[r'Cate_Label'].isin(_selected_cates)]

        # remove those rows with any missing value
        df = df.dropna()

        # step 3: transform data to the form we need
        # replace category name (str) with label number (int)
        f = lambda cate: {
                r'ツール類': 0,
                r'ソーシャルネットワーキング': 1,
                r'ゲーム': 2
            }[cate]
        df['Cate_Label'] = df['Cate_Label'].map(f, meta=('Cate_Label', int))

        # generate date columns
        df['Date'] = df['Timestamp'].map(lambda x: x[:10], meta=('Date', str))

        # convert time (str dtype) to timestamp (int dtype), then divide 10^9
        df['Timestamp'] = df['Timestamp'].astype('M8[us]').\
            astype(np.int64) // 10 ** 9

        # step 4: generate ret data as result
        # split original df into 3 categories            
        f = lambda x: x.tolist()
        
        for i in range(3):  # 因為是3個類別, 故range == 3
            print(src_file_nm, i)  # for testing usage
            
            _dfc = df[df['Cate_Label'] == i]
            _dfc_gr = _dfc.groupby(['Monitor_ID', 'Date'])
            
            _dfc_gr['Timestamp']\
                .apply(f, meta=('Timestamp', int))\
                .compute()\
                .to_csv(os.path.join(dest_folder_pa,
                                     f'{src_file_nm[:-4]}_{i}_ts.csv'))
            _dfc_gr['Duration']\
                .apply(f, meta=('Duration', int))\
                .compute()\
                .to_csv(os.path.join(dest_folder_pa, 
                                     f'{src_file_nm[:-4]}_{i}_dr.csv'))

#     def preprocess_all(self) -> None:
#         '''
#             前處理主程序在這裡
#         '''
#         file_nm_ls = [fl for fl in os.listdir(self._project_root) 
#                       if fl.endswith('.tsv')]
#         file_nm_ls.sort()

#         for fl_nm in file_nm_ls:
#             fl_pa = os.path.join(self._project_root, fl_nm)
#             fl_pa_prefix = fl_pa.split('.')[0]
            
#             print(f'開始處理檔案 {fl_nm} \n>> 正在進行前處理中...')
#             pre_str = r'>> 正在計算並儲存前處理結果: '
#             dfc_ts, dfc_dr = issp._preprocess(fl_pa)
            
#             print(pre_str + r'generate login timestamp time series & save data...')
#             ret_dfc_ts, file_pa = self._compute(dfc_ts), fl_pa_prefix + r'_ts.json'
#             self._save_file(ret_dfc_ts, file_pa)
#             del ret_dfc_ts
            
#             print(pre_str + r'generate duration time series & save data...')
#             ret_dfc_dr, file_pa = self._compute(dfc_dr), fl_pa_prefix + r'_dr.json'
#             self._save_file(ret_dfc_dr, file_pa)
#             del ret_dfc_dr

#             print(f'檔案 {fl_nm} 已處理完成')
            
#     def _load_json(self, jsonfilename: str) -> dict:
#     def _load_json(self, jsonfile_pa: str) -> dict:
#         with gzip.GzipFile(jsonfile_pa, 'r') as fin:
#             return json.loads(fin.read().decode('utf-8'))
        
#     def extract_feature_all(self) -> None:
#         def _ts_feature(ts) -> List[float]:
#             _max, _min, _count = max(ts), min(ts), len(ts)
#             _len = _max - _min
#             _den = _len / _count
#             return [_max, _min, _len, _count, _den]
        
#         def _dr_feature(dr) -> List[float]:
#             _max, _min = max(dr), min(dr)
#             _mean, _median, _std = sum(dr) / len(dr), np.median(dr).item(), np.std(dr).item()
#             return [_max, _min, _mean, _median, _std]
        
#         # columns name of csv data frame
#         _df_cols = [
#             r'max(ts)',
#             r'min(ts)',
#             r'max(ts)-min(ts)=diff(ts)',
#             r'len(ts)',
#             r'diff(ts)/len(ts)',
#             r'max(dr)',
#             r'min(dr)',
#             r'mean(dr)',
#             r'median(dr)',
#             r'var(dr)',
#         ]
        
#         f = lambda postfix: [fl for fl in os.listdir(self._project_root) if fl.endswith(postfix)]
#         ts_file_ls, dr_file_ls = f('_ts.json'), f('_dr.json')
#         ts_file_ls.sort(); dr_file_ls.sort()
        
#         for ts_fl, dr_fl in zip(ts_file_ls, dr_file_ls):
#             print(f'{ts_fl} {dr_fl}')
#             d_ts, d_dr = self._load_json(ts_fl), self._load_json(dr_fl)
            
#             for i in ['0', '1', '2']:
#                 print(i)
#                 L, ret = len(d_ts[i]), []
                
#                 for j in range(L):
#                     tmp = _ts_feature(d_ts[i][j])
#                     tmp.extend(_dr_feature(d_dr[i][j]))
#                     ret.append(tmp)
                
#                 # save as pandas dataframe
#                 file_nm = ts_fl[:-7] + i + r'.csv'
#                 df = pd.DataFrame(ret, columns=_df_cols, dtype=float)
#                 df.to_csv(os.path.join(self._project_root, file_nm), index=False)
                
#     def _load_csv_all(self) -> Tuple['numpy.array']:
#         ret_X, ret_Y = np.array([]).reshape((-1, 10)), np.array([])
#         file_ls = [fl for fl in os.listdir(self._project_root) 
#                    if fl.endswith('.csv')]
        
#         for fl in file_ls:
#             print(f'>> Loading {fl} ...')
#             cate = int(fl[-5])
#             arr_X = pd.read_csv(os.path.join(self._project_root, fl))
#             ret_X = np.concatenate((ret_X, arr_X))
#             arr_Y = np.array([cate] * arr_X.shape[0], dtype=np.int64)
#             ret_Y = np.concatenate((ret_Y, arr_Y))
        
#         return ret_X, ret_Y
    
#     def train(self) -> None:
#         # refer this article: https://www.datacamp.com/community/tutorials/random-forests-classifier-python        
#         # about stratify: https://michaeljsanders.com/2017/03/24/stratify-continuous-variable.html
#         # split train data and testing data
#         X, Y = self._load_csv_all()
#         Xr, Xt, Yr, Yt = train_test_split(X, Y, test_size=self._test_size, 
#                                           stratify=Y, random_state=None)
        
#         # create model
#         self._clf = TPOTClassifier(**tpot_config)
        
#         # training
#         self._clf.fit(Xr, Yr)
        
#         # predict testing data
#         Yp = self._clf.predict(Xt)
        
#         # calculate loss
#         print(self._clf.score(Xt, Yt))
        
#         # print confusion matrix
#         arr_conf = confusion_matrix(Yt, Yp)
#         print(arr_conf)

In [5]:
df = dd.read_csv(r'D:\issp_data\raw_user\s35809_MobileApp_201701_104628_user.tsv', sep='\t')
print(df.shape)
df.head(50)

(Delayed('int-d9fe0fda-4477-4ec6-9ee8-8d5c242b7009'), 11)


,年月,モニター登録コード ※納品不可,モニターSCI ※納品不可,モニターID 社外提供 ※ローデータ提供以外不可,HHID ※納品不可,TVエリア,PC,Mobile,TV,クロスメディア区分,ウェイト
0,2017-01,00000feac1cd8a64c4631ee7567e74ea581ac848,9400103854,9X89NAK526DCVSHN,2.256795e+11,関東,1,1,0,トータル,3.187680
1,2017-01,0005992f11d890079ad67eaa90b15651d7fd4505,9500028729,BZF5AGT42H5TZAA3,3.436422e+11,関東,1,1,1,トータル,3.709412
2,2017-01,000aeb73439531ac09cab2c43b1542a2033b6f4d,9500033427,TQ3AJTH7732QLMYH,2.866259e+11,その他,1,1,0,トータル,4.732195
3,2017-01,000e6695405fa8f1911da244a9388ec2302ddef2,9500029713,RWS9G6WCMHXVUXPQ,NaN,関東,0,1,0,トータル,4.254513
4,2017-01,00140443bd6c1c7cd43d5113cfe28791717068ef,9400102187,7Z5F9XWCNFZRVWW9,NaN,関東,0,1,0,トータル,2.468148
5,2017-01,0016d8353d40ebbac555f584c21e5d5f866c81dc,9400002467,6NVSA3H79UPA73YZ,8.392017e+11,その他,1,1,0,トータル,4.085743
6,2017-01,0020dcd8843a4967d6d9ed044b712de491bb0d23,9500038174,MUASTUU66UFF3PSN,NaN,関西,0,1,0,トータル,1.769911
7,2017-01,00218addedad65baf7f07b0ef3ff00828c5b598f,9400040000,3VUU6WNAGBHV7JYD,9.824440e+11,その他,1,1,0,トータル,3.429736
8,2017-01,00273635df7efde412544500e6e51d098a5640bd,9500015262,JSNZ43S9CAP8669T,1.712923e+11,その他,1,1,0,トータル,2.541545
9,2017-01,0033a89fa35ae003e98e6efef3554b2da30a0148,9500025625,HUWRB5Z9C8RV64UX,NaN,関西,0,1,0,トータル,2.959927


In [6]:
df = dd.read_csv(r'D:\issp_data\raw_tsdr\s35809_MobileApp_201701_104628.tsv', sep='\t')
print(df.shape)
df.head(50)

(Delayed('int-c3b09200-bc3c-4fa6-b116-c21594cfbd44'), 10)


,モニターCue ※納品不可,日時(yyyy-mm-dd hh:mm:ss),接触時間（duration),OS（Android or iOS）,コンテンツID,パッケージId,コンテンツ名,アプリカテゴリ,アプリ提供企業名,アプリ名
0,50000039179,2017-01-03 20:16:26,3,iOS,565,jp.co.agoop.SetsuzokuRadarRelase,電波つながりチェッカー,ツール類,AGOOP corp.,電波つながりチェッカー
1,50000039179,2017-01-03 20:15:32,2,iOS,565,jp.co.agoop.SetsuzokuRadarRelase,電波つながりチェッカー,ツール類,AGOOP corp.,電波つながりチェッカー
2,50000039179,2017-01-03 20:12:32,3,iOS,565,jp.co.agoop.SetsuzokuRadarRelase,電波つながりチェッカー,ツール類,AGOOP corp.,電波つながりチェッカー
3,50000039179,2017-01-03 20:11:38,2,iOS,565,jp.co.agoop.SetsuzokuRadarRelase,電波つながりチェッカー,ツール類,AGOOP corp.,電波つながりチェッカー
4,50000039179,2017-01-03 20:09:57,1,iOS,565,jp.co.agoop.SetsuzokuRadarRelase,電波つながりチェッカー,ツール類,AGOOP corp.,電波つながりチェッカー
5,50000039179,2017-01-03 20:06:25,2,iOS,565,jp.co.agoop.SetsuzokuRadarRelase,電波つながりチェッカー,ツール類,AGOOP corp.,電波つながりチェッカー
6,50000039179,2017-01-03 20:05:32,3,iOS,565,jp.co.agoop.SetsuzokuRadarRelase,電波つながりチェッカー,ツール類,AGOOP corp.,電波つながりチェッカー
7,50000039179,2017-01-03 20:04:21,2,iOS,565,jp.co.agoop.SetsuzokuRadarRelase,電波つながりチェッカー,ツール類,AGOOP corp.,電波つながりチェッカー
8,50000039179,2017-01-03 20:03:50,1,iOS,565,jp.co.agoop.SetsuzokuRadarRelase,電波つながりチェッカー,ツール類,AGOOP corp.,電波つながりチェッカー
9,50000039179,2017-01-03 19:44:56,2,iOS,565,jp.co.agoop.SetsuzokuRadarRelase,電波つながりチェッカー,ツール類,AGOOP corp.,電波つながりチェッカー


In [4]:
df = dd.read_csv(r'D:\issp_data\s35809_extract_condition_dl_list_104628.tsv', sep='\t')
print(df.shape)
df.head(10)

(Delayed('int-5a4174fd-aab4-4d24-98f8-a0ebd773e4d3'), 58)


C:\Anaconda3\lib\site-packages\dask\dataframe\core.py:5721: UserWarning: Insufficient elements for `head`. 10 elements requested, only 1 elements available. Try passing larger `npartitions` to `head`.
  warnings.warn(msg.format(n, len(r)))


,タイトル,集計開始日,集計終了日,日付定義,抽出データ,有効判定区分,積上／継続,データ抽出条件(WEB[PC]),接触秒数(WEB[PC]),サイト除外条件(WEB[PC]),...,ウェイト選択(新/旧),モニター有効メディア区分,ウェイト母集団,ブラックリストモニター除外,TVエリア,OSバージョン除外(PC),OSバージョン除外(Android),OSバージョン除外(iOS),出力項目,出力ファイル
0,MobileApp_201701,2017-01-01,2017-01-31,1:0-24時,APP(MB),1:確定（月次）,1:積上,NaN,NaN,NaN,...,0:新ウェイト,2:MBTotal,0:インターネット利用人口,1:除外する,"1111:関東,関西,中京,その他",しない,する,する,"7:モニターCue ※納品不可,99:日時(yyyy-mm-dd hh:mm:ss),90:...","設定条件ファイル,有効モニターリスト,集計(抽出)結果"


In [61]:
issp = ISSP()
issp.preprocess_all()

['s35809_MobileApp_201701_104628.tsv', 's35809_MobileApp_201702_104645.tsv', 's35809_MobileApp_201703_104646.tsv', 's35809_MobileApp_201704_104647.tsv', 's35809_MobileApp_201705_104648.tsv', 's35809_MobileApp_201706_104649.tsv', 's35809_MobileApp_201707_104650.tsv', 's35809_MobileApp_201708_104651.tsv', 's35809_MobileApp_201709_104652.tsv', 's35809_MobileApp_201710_104653.tsv', 's35809_MobileApp_201711_104654.tsv', 's35809_MobileApp_201712_104655.tsv']
s35809_MobileApp_201701_104628.tsv 0


C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:103: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:108: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.


s35809_MobileApp_201701_104628.tsv 1


KeyboardInterrupt: 

In [12]:
rr1, rr2 = ret

In [15]:
re11 = rr1[0].compute()

In [16]:
re11

Monitor_ID   Date      
50000010155  2017-01-01    [1483313244, 1483312912, 1483313391, 148331342...
50000011091  2017-01-16    [1484551003, 1484551182, 1484551162, 148455115...
50000011483  2017-01-04    [1483517589, 1483517712, 1483519798, 148351978...
50000011502  2017-01-21    [1485031656, 1485026360, 1485028774, 148502880...
50000012191  2017-01-23    [1485149232, 1485149282, 1485208484, 148516526...
50000012533  2017-01-29    [1485692549, 1485728503, 1485692409, 148569253...
50000012924  2017-01-14    [1484433445, 1484354060, 1484354129, 148435423...
50000012953  2017-01-06    [1483724012, 1483724052, 1483724002, 148373905...
50000013074  2017-01-03    [1483408931, 1483414052, 1483413984, 148341374...
             2017-01-04    [1483560476, 1483560349, 1483560343, 148351822...
50000013189  2017-01-06    [1483743153, 1483743130, 1483688564, 148368852...
50000013194  2017-01-13     [1484340632, 1484317561, 1484298208, 1484340622]
50000013296  2017-01-05                             

In [17]:
type(re11)

pandas.core.series.Series

In [27]:
re11.to_csv(r'D:\issp_data\test.csv')

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [25]:
jsonfile_pa = r'D:\issp_data\ts_json\s35809_MobileApp_201701_104628_ts.json'
js = issp._load_json(jsonfile_pa)

In [26]:
js

{'0': [[1483313244,
   1483312912,
   1483313391,
   1483313429,
   1483231459,
   1483231728,
   1483232831,
   1483233078,
   1483233911,
   1483234780,
   1483298116,
   1483311745,
   1483308465,
   1483309397,
   1483309311,
   1483309081,
   1483311969,
   1483231430,
   1483263578,
   1483271504,
   1483311274,
   1483312468,
   1483312344,
   1483312404,
   1483312789,
   1483286907,
   1483272969,
   1483281697,
   1483281527,
   1483281557,
   1483281604,
   1483263544,
   1483281614,
   1483281625,
   1483281680,
   1483308380,
   1483304018,
   1483262866,
   1483301865,
   1483287545,
   1483287599,
   1483231307,
   1483286498,
   1483298091,
   1483263677,
   1483282716,
   1483281708,
   1483263604,
   1483272953,
   1483272918,
   1483272638,
   1483272237,
   1483287705,
   1483288111,
   1483281524,
   1483271262,
   1483262823,
   1483262786,
   1483271293,
   1483271442,
   1483271488,
   1483263045,
   1483262859,
   1483263353,
   1483263538,
   1483262120,
   14

In [4]:
if __name__ == '__main__':
    proj_config = {
        r'project_root': r'D:\issp_data\rawdata\s35809_MobileApp_201701_104628.tsv',
        r'test_size': 0.2
#         r'project_root' = r'/content/drive/My Drive/2017_issp_project/feature_csv'
    }
    tpot_config = {
        r'generations': 5,
        r'population_size': 250,
        r'verbosity': 3,
        r'cv': 5,
        r'subsample': 0.01,
#         r'n_jobs': 1
        r'n_jobs': -1
    }
    
    issp = ISSP(proj_config, tpot_config)
#     issp.preprocess_all()
#     issp.extract_feature_all()
    t0 = time.time()
    issp.train()
    print(r'total excution time', time.time() - t0)

>> Loading s35809_MobileApp_201701_104628_0.csv ...
>> Loading s35809_MobileApp_201701_104628_1.csv ...
>> Loading s35809_MobileApp_201701_104628_2.csv ...
>> Loading s35809_MobileApp_201702_104645_0.csv ...
>> Loading s35809_MobileApp_201702_104645_1.csv ...
>> Loading s35809_MobileApp_201702_104645_2.csv ...
>> Loading s35809_MobileApp_201703_104646_0.csv ...
>> Loading s35809_MobileApp_201703_104646_1.csv ...
>> Loading s35809_MobileApp_201703_104646_2.csv ...
>> Loading s35809_MobileApp_201704_104647_0.csv ...
>> Loading s35809_MobileApp_201704_104647_1.csv ...
>> Loading s35809_MobileApp_201704_104647_2.csv ...
>> Loading s35809_MobileApp_201705_104648_0.csv ...
>> Loading s35809_MobileApp_201705_104648_1.csv ...
>> Loading s35809_MobileApp_201705_104648_2.csv ...
>> Loading s35809_MobileApp_201706_104649_0.csv ...
>> Loading s35809_MobileApp_201706_104649_1.csv ...
>> Loading s35809_MobileApp_201706_104649_2.csv ...
>> Loading s35809_MobileApp_201707_104650_0.csv ...
>> Loading s



TPOT closed during evaluation in one generation.


TPOT closed prematurely. Will use the current best pipeline.
0.5313876628775541
[[823026  18677      0]
 [426004  59602      0]
 [301342  32336      0]]
total excution time 53.86225199699402
